In [1]:
import numpy as np
import tensorflow as tf

import os

In [37]:
TRAIN_TFRECORD_FILENAME = "train.tfrecord"
VALID_TFRECORD_FILENAME = "valid.tfrecord"
DATA_DIR = "data"
BUFFER_SIZE = 64
BATCH_SIZE = 64
SECTION_SIZE = 44100 // 4

# Load Record

In [59]:
train_dataset = tf.data.TFRecordDataset([os.path.join(DATA_DIR, TRAIN_TFRECORD_FILENAME)])
valid_dataset = tf.data.TFRecordDataset([os.path.join(DATA_DIR, VALID_TFRECORD_FILENAME)])

In [69]:
feature_description = {
    'waveform': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'label': tf.io.FixedLenFeature([], tf.float32, default_value=0.0)
}

def _parse_function(example_proto):
    example = tf.io.parse_single_example(example_proto, feature_description)
    waveform = tf.io.parse_tensor(example['waveform'], out_type=tf.float32)
    waveform = tf.ensure_shape(waveform, (SECTION_SIZE, 2))
    label = tf.ensure_shape(example['label'], ())
    return waveform, label

In [70]:
train_dataset = train_dataset.map(_parse_function)
valid_dataset = valid_dataset.map(_parse_function)

In [71]:
for i in train_dataset.take(1):
    print(i)

(<tf.Tensor: shape=(11025, 2), dtype=float32, numpy=
array([[-0.08157349, -0.07055664],
       [-0.07818604, -0.07647705],
       [-0.06790161, -0.07702637],
       ...,
       [-0.04818726,  0.04501343],
       [-0.04989624,  0.059021  ],
       [-0.04837036,  0.07107544]], dtype=float32)>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>)


# Preparation

In [72]:
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(BUFFER_SIZE).cache()
valid_dataset = valid_dataset.prefetch(BUFFER_SIZE).cache()

# First Model

In [80]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(10),
    tf.keras.layers.Dense(1, activation="softmax")
])

In [81]:
model1.compile(optimizer='adam', loss='mse')

In [ ]:
model1.fit(train_dataset, epochs=3, validation_data=valid_dataset)

Epoch 1/3
     55/Unknown - 244s 4s/step - loss: 0.7838

In [82]:
model1.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.